In [ ]:
import pandas as pd
import boto3
import json
import numpy as np

### Load Redshift params from config file

In [ ]:
import configparser
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

KEY                    = config.get('AWS','KEY')
SECRET                 = config.get('AWS','SECRET')

DWH_CLUSTER_IDENTIFIER = config.get("DWH","DWH_CLUSTER_IDENTIFIER")
DWH_DB                 = config.get("CLUSTER","DB_NAME")
DWH_DB_USER            = config.get("CLUSTER","DB_USER")
DWH_DB_PASSWORD        = config.get("CLUSTER","DB_PASSWORD")
DWH_PORT               = config.get("CLUSTER","DB_PORT")

#### Create clients

In [ ]:
redshift = boto3.client('redshift',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                       )

#### Check cluster status

In [ ]:
def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', None)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

#### Get the endpoint and ARN

In [ ]:
DWH_ENDPOINT = myClusterProps['Endpoint']['Address']
DWH_ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']
endpoint = 'dwhcluster.c7mzactav3lc.us-west-2.redshift.amazonaws.com'
roleArn = 'arn:aws:iam::489967615225:role/dwhRole'
print("DWH_ENDPOINT :: ", endpoint)
print("DWH_ROLE_ARN :: ", roleArn)

### Connect to Redshift

In [ ]:
%load_ext sql

In [ ]:
conn_string="postgresql://{}:{}@{}:{}/{}".format(DWH_DB_USER, DWH_DB_PASSWORD, DWH_ENDPOINT, DWH_PORT,DWH_DB)
print(conn_string)
%sql $conn_string

#### Show all tables

In [ ]:
%sql SELECT t.table_name FROM information_schema.tables t \
WHERE t.table_schema = 'public' AND t.table_type = 'BASE TABLE' \
ORDER BY t.table_name;

#### How many fact records?

In [ ]:
%sql SELECT COUNT(*) FROM sensorevents;

### Get the top 10 groups with highest average body temperature

In [ ]:
%%sql SELECT g.group_name, AVG(se.body_temperature) AS avg_temp 
FROM groups g INNER JOIN sensorevents se ON g.group_id = se.group_id 
GROUP BY g.group_name 
ORDER BY  avg_temp DESC 
LIMIT 10;

### Find cows with the top five average rumination in each group

In [ ]:
%%sql WITH avg_r AS ( 
    SELECT c.cow_number, c.cow_name, se.group_id AS group_id, AVG(se.rumination) AS avg_rumination 
    FROM sensorevents se INNER JOIN cows c ON se.cow_number = c.cow_number 
    GROUP BY c.cow_number, c.cow_name, se.group_id 
) 
SELECT ar.cow_number, ar.cow_name, g.group_name, ar.avg_rumination, r_rank 
FROM (SELECT *, RANK() OVER (PARTITION BY group_id ORDER BY  avg_rumination DESC) AS r_rank 
FROM avg_r) AS ar 
INNER JOIN groups g ON g.group_id = ar.group_id 
WHERE r_rank <= 5 
ORDER BY g.group_name, ar.avg_rumination DESC;

### Get average and standard deviation of rumination by group

In [ ]:
body_temps = %sql SELECT g.description AS group, AVG(e.rumination) AS avg, STDDEV(e.rumination) \
FROM sensorevents e INNER JOIN groups g on e.group_id = g.group_id \
GROUP BY g.description \
ORDER BY avg

df = body_temps.DataFrame()
df

In [ ]:
import matplotlib.pyplot as plt

df.plot(x="group", y=["avg", "stddev"])